## Our own transformer design
The [Unibalun](https://github.com/DG1JAN/UniBalun) can serve as an example to design our own transformer.
DL2MAN has done some [measurements on this and similar impedance transformers](https://www.youtube.com/watch?v=nNzTf1F12BE).

### Calculation : 
Given parameters:
* source impedance = 50ohm
* turns ratio = 1:7 (will result in 1:49 impedance ratio)
* Max power = 5W
* Frequency range = 3-30MHz

In [2]:
import math
Zin = 50 # [ohm] source impedance
fmin = 3e6 # Lowest usable frequency[Hz]
turns_ratio = 7
Pmax = 5 # [W]

# Calculate impedance of primary coil
# Impedance of primary winding must be greater than 5x source impedance.
Xp_min = 5 * Zin

# Calculate inductance of primary coil
Lp_min = Xp_min / (2*math.pi*fmin)
print('Minimum required inductance is {:.3}H'.format(Lp_min))

Minimum required inductance is 1.33e-05H


### Core sizes

If a core material has been chosen, the inductance is determined by the core geometry : cross-sectional (CSA) area and diameter.
The higher the CSA and smaller the diameter, the lesser turns needed to reach a certain inductance value.

We generally prefer less turns to reduce losses and for ease of winding.

### Core materials
[Fair-rite materials](https://www.fair-rite.com/materials/): Inductive Materials : 
 * [#43](https://www.fair-rite.com/43-material-data-sheet/) < 10MHz, permeability is very temperature dependent
 * [#61](https://www.fair-rite.com/61-material-data-sheet/) < 25MHz, much more stable permeability with respect to temperature.
 * [SIFERRIT K1](https://www.tdk-electronics.tdk.com/download/528860/ec9323ccb94c129f0da1c855d7963b2d/pdf-k1.pdf) : < 20MHz
 * [#67](https://www.fair-rite.com/67-material-data-sheet/) < 50MHz
 
#### Using a #43-core
This core is not suited for this frequency range according to the specs of the core material.  Let's try it anyway because it seems to be so common in the ham HF-world.

* [Fair-Rite 5943001301](https://www.fair-rite.com/product/toroids-5943001301/)
  * [Measurement results of stacked core](./Measurement_Results/Fair-Rite_5943001301.ipynb)
* Probably the best core for QRP : [Fair-Rite 2643625002](https://www.fair-rite.com/product/round-cable-emi-suppression-cores-2643625002/) : Al = 2910nH/N²
   * [Evaluating efficiency, transformer and mismatch losses in toroid transformers - some observations.](https://www.ai6xg.com/post/evaluating-efficiency-transformer-and-mismatch-losses-in-toroid-transformers-some-observations)
   * [Small efficient matching transformer for an EFHW](https://owenduffy.net/blog/?p=12642)
   * to be tested with 3:21 windings
* Alternative cores : 
   * [Amidon FT-50-43](https://www.amidoncorp.com/ft-50-43/) : to be tested with 3:27 windings
   * [Amidon FT-140-43](https://www.amidoncorp.com/ft-140-43/) = [Fair-Rite 2643802702](https://www.fair-rite.com/product/round-cable-emi-suppression-cores-2643802702/)
   * [Amidon FT-114-43](https://www.amidoncorp.com/ft-114-43/), as used on [Engineering the EFHW 49:1 Transformer and Antenna](https://squashpractice.com/2021/07/20/engineering-the-efhw-491-transformer-and-antenna/)


#### Using a SIFERRIT K1-core
[SIFERRIT K1 measurements](./Measurement_Results/SIFERRIT_K1.ipynb)
 
#### Using 4 W 620-core
 * [WE 7427010](https://www.we-online.com/components/products/datasheet/7427010.pdf)
 * [Measurements on WE 74270023](./Measurement_Results/WE_74270023.ipynb)

#### Using a 3 W 180-core
 * [WE 7427056](https://www.we-online.com/components/products/datasheet/7427056.pdf)
 * [Measurements](./Measurement_Results/WE_7427056.ipynb)
 
#### Using a #61-core
 * [Amidon FT 82-61  = Fair-rite 5961000601](https://www.digikey.be/en/products/detail/fair-rite-products-corp/5961000601/8594131) : ⌀21mm, #61, €1.69
 * [Fair-rite 5961001821](https://www.digikey.be/en/products/detail/fair-rite-products-corp/5961001821/15930539) : ⌀23mm, #61, €1.42, ⌀23mm

 [#61-core measurements](./Measurement_Results/type_61_material.ipynb)
 
#### Using a #67-core
Let's select a 67-core because it matches the required frequency range: [FT87-67](https://www.amidoncorp.com/ft-87-67/). 

160 windings is too much to wind on such a small FT-67-87 core.  No core has been bought for this frequency range.

 ### Calculations
 * [Miguel VK3CPU - RF Toroid Calculator v1.0](https://miguelvaca.github.io/vk3cpu/toroid.html)- : don't use the wire length calculator from this site as it yields much exxagerated lengths.
 * [Toroid Winding Calculator](http://www.calculatoredge.com/electronics/coilwinding.htm#toroid)

In [3]:
import sys
sys.path.insert(0, './Calculations')
from calcInductors import *

d = 0.5e-3 # [m] wire diameter

#Amidon FT-87-67 (#67 material)
# Toroid
Al = 25 # [nH/N²] Inductance factor
Ae = 0.2612e-4 # [m²] Effective cross-sectional area
Bmax = 0.23 # [T], from [Fair-rite materials](https://www.fair-rite.com/materials/): Inductive Materials
ID = 13.7e-3 # [m], inner diameter
OD = 22e-3 # [m], outer diameter
h = 6.35e-3 # [m], height
  
[Np, Ns] = calc_inductor_turns(Al, Lp_min, turns_ratio)
print("Required number of turns on primary winding: " + str(Np))
print("Required number of turns on secondary winding: " + str(Ns))
if Np+Ns < calc_toroid_max_turns(ID, d):
    print("The total number of turns will fit on the inductor.")
else:
    print("The total number of turns doesn't fit on the inductor.")

print("Primary coil wire length: " + str(calc_toroid_wirelength(OD, ID, d, h, Np))+"m")
print("Secondary coil wire length: " + str(calc_toroid_wirelength(OD, ID, d, h, Ns))+"m")

B = calc_max_flux_density(Ae, Pmax, Zin, fmin, Np)
print('Maximum flux density: {:.3}T'.format(B))
if B > Bmax:
    print("Error : Exceeding maximum flux density.")
else:
    print("Flux density OK.")


Required number of turns on primary winding: 24
Required number of turns on secondary winding: 168
The total number of turns doesn't fit on the inductor.
Primary coil wire length: 0.52m
Secondary coil wire length: 4.22m
Maximum flux density: 0.00189T
Flux density OK.


## References
* ARRL Handbook 2018, §5.7.3 Broadband Ferrite RF Transformers
* Secrets of RF Circuit Design, Joseph J. Carr, §4. Winding your own coils
* [Steve Dick, K1RF : The end-fed half-wave antenna](http://gnarc.org/wp-content/uploads/The-End-Fed-Half-Wave-Antenna.pdf)
* [Performance of 49:1 ferrite core transformers](https://squashpractice.com/2021/06/23/performance-of-491-ferrite-core-transformers/)
* [KN5L EFHW Unun](https://www.kn5l.net/kn5lEfhwUnun/)
* [Engineering the EFHW 49:1 Transformer and Antenna](https://squashpractice.com/2021/07/20/engineering-the-efhw-491-transformer-and-antenna/)


## Ideas for the future
We could make a transformer, suitable for EFHW and dipole.  Three coils 1:4:5 coil ratio.
* EFHW  = 1 : (4+5) = 1:9
* Dipole = (1+4) : 5 = 5:5